In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# 1 - Develop CNN and Resnet50

In [11]:
# Training data generator with augmentation
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True)

# Test data generator (no augmentation, just rescale)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training dataset
train_generator = train_datagen.flow_from_directory(
    'train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Load the testing dataset
test_generator = test_datagen.flow_from_directory(
    'test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Don't shuffle so the images are in the same order for evaluation
)

Found 750 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


## Model definition

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential()

# 1st Convolutional Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Convolutional Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Convolutional Block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the convolutional outputs into a 1D vector
model.add(Flatten())

# Fully connected Dense layer with 10 neurons
model.add(Dense(10, activation='relu'))

# Output layer with Softmax activation for binary classification
model.add(Dense(2, activation='softmax'))  # 2 neurons for "rust" and "no rust"

# Compile the model
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Print model summary
model.summary()


c:\Users\quang\src\uni\2024-sem2\cos40007\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │       369,930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 463,200 (1.77 MB)

 Trainable params: 463,200 (1.77 MB)

 Non-trainable params: 0 (0.00 B)

## Model training

In [5]:
checkpoint = ModelCheckpoint('best_model.keras', 
                             monitor='val_accuracy', 
                             save_best_only=True, 
                             mode='max', 
                             verbose=1)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    callbacks=[checkpoint]
)


Epoch 1/10


c:\Users\quang\src\uni\2024-sem2\cos40007\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 475ms/step - accuracy: 0.5298 - loss: 0.8260
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to best_model.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 17s 515ms/step - accuracy: 0.5290 - loss: 0.8228 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 2/10
 1/23 ━━━━━━━━━━━━━━━━━━━━ 5s 256ms/step - accuracy: 0.5625 - loss: 0.6924
Epoch 2: val_accuracy did not improve from 0.50000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5625 - loss: 0.6924 - val_accuracy: 0.5000 - val_loss: 0.6929
Epoch 3/10


C:\Users\quang\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 0.5344 - loss: 0.6925
Epoch 3: val_accuracy did not improve from 0.50000
23/23 ━━━━━━━━━━━━━━━━━━━━ 10s 361ms/step - accuracy: 0.5352 - loss: 0.6925 - val_accuracy: 0.5000 - val_loss: 0.6839
Epoch 4/10
 1/23 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - accuracy: 0.6250 - loss: 0.6774
Epoch 4: val_accuracy did not improve from 0.50000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6250 - loss: 0.6774 - val_accuracy: 0.5000 - val_loss: 0.6854
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.5887 - loss: 0.6678
Epoch 5: val_accuracy improved from 0.50000 to 0.65000, saving model to best_model.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 388ms/step - accuracy: 0.5892 - loss: 0.6675 - val_accuracy: 0.6500 - val_loss: 0.6209
Epoch 6/10
 1/23 ━━━━━━━━━━━━━━━━━━━━ 5s 268ms/step - accuracy: 0.5938 - loss: 0.6341
Epoch 6: val_accuracy improved from 0.65000 to 0.80000, saving model to best_model.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/s

## Evaluate on test set

In [8]:
# Load the best model
model.load_weights('best_model.keras')

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.9000 - loss: 0.3287
Test accuracy: 90.00%


## Make Predictions on Test Images

In [9]:
import numpy as np
predictions = model.predict(test_generator)
predicted_classes = np.where(predictions > 0.5, 1, 0)  # Classify as 1 (rust) or 0 (no-rust)

# Compare with actual labels
true_classes = test_generator.classes
accuracy_per_image = np.sum(predicted_classes == true_classes) / len(true_classes)
print(f"Accuracy on individual test images: {accuracy_per_image * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Accuracy on individual test images: 1000.00%
